<a href="https://colab.research.google.com/github/2SUBDA/Block2_Walk/blob/master/PredSampleForARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# MODEL - Save finalized model
# Logic based on DeptOneTimeSeries.ipynb sample 

# monkey patch around bug in ARIMA class
def __getnewargs__(self):
    return ((self.endog),(self.k_lags, self.k_diff, self.k_ma))

ARIMA.__getnewargs__ = __getnewargs__

# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return diff

# Load the dataset for modeling
series = Series.from_csv('dataset.csv', header = 0)

# prepare data
X = series.values
X = X.astype('float32')

# difference data
months_in_year = 12
diff = difference(X, months_in_year)

# fit model
model = ARIMA(diff, order=(6, 2, 1)) # Using the best RMSE
model_fit = model.fit(trend='nc', disp=0)

# bias constant, could be calculated from in-sample mean residual
bias = 74.2230 # Using the residual model mean as our bias

# save model
model_fit.save('model.pkl')
np.save('model_bias.npy', [bias])

In [0]:
# Interpret - make one prediction
# Logic based on DeptOneTimeSeries.ipynb sample

def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# Load the dataset for prediction
series = Series.from_csv('dataset.csv', header = 0)

months_in_year = 12
model_fit = ARIMAResults.load('model.pkl')
bias = np.load('model_bias.npy')
yhat = float(model_fit.forecast()[0])
yhat = bias + inverse_difference(series.values, yhat, months_in_year)
print('Predicted: %.3f' % yhat)

In [0]:
# MODEL - Validating Forecasts
# Logic based on DeptOneTimeSeries.ipynb sample 

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return diff

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# load and prepare datasets
dataset = Series.from_csv('dataset.csv', header = 0)
X = dataset.values.astype('float32')
history = [x for x in X]
months_in_year = 12
validation = Series.from_csv('validation.csv', header = 0)
y = validation.values.astype('float32')



In [0]:
# MODEL - Make forward predictions

# load model
model_fit = ARIMAResults.load('model.pkl')
bias = np.load('model_bias.npy')

# make first prediction
predictions = list()
yhat = float(model_fit.forecast()[0])
yhat = bias + inverse_difference(history, yhat, months_in_year)
predictions.append(yhat)
history.append(y[0])
print('>Predicted=%.3f, Expected=%3.f' % (yhat, y[0]))

# rolling forecasts
for i in range(1, len(y)):
	# difference data
	months_in_year = 12
	diff = difference(history, months_in_year)
	# predict
	model = ARIMA(diff, order=(6, 2, 1)) # Using the mest ARIMA
	model_fit = model.fit(trend='nc', disp=0)
	yhat = model_fit.forecast()[0]
	yhat = bias + inverse_difference(history, yhat, months_in_year)
	predictions.append(yhat)
	# observation
	obs = y[i]
	history.append(obs)
	print('>Predicted=%.3f, Expected=%3.f' % (yhat, obs))



In [0]:
# Interpret - Save and Plot

pred_dates = ['2014-01','2014-02', '2014-03']
# Send the prediction to csv file    
#predictions.to_csv('predictions.csv')

# report performance
rmse = sqrt(mean_squared_error(y, predictions))
print('RMSE: %.3f' % rmse)
plt.plot(y)
plt.plot(predictions, color='red')
plt.title('Predicted Median Home Price vs Actual Home Price')
plt.ylabel('Median Home Price')
plt.show()